In [4]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import openpyxl

In [5]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

2.2.3
3.4.2


In [11]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time



In [12]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(4)

ImportError: Missing optional dependency 'xlrd'. Install xlrd >= 2.0.1 for xls Excel support Use pip or conda to install xlrd.

In [ ]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

In [ ]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

In [ ]:
# enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

In [ ]:
# running the model through EMA workbench
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=50, policies=4)

In [ ]:
# observing the simulation runs
experiments, outcomes = results
print(outcomes.keys())
experiments

In [ ]:
# only works because we have scalar outcomes
#pd.DataFrame(outcomes)
outcomes

In [ ]:
# defining specific policies
# for example, policy 1 is about extra protection in upper boundary
# policy 2 is about extra protection in lower boundary
# policy 3 is extra protection in random locations


def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}


policies = [
    Policy(
        "policy 1",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "A.1_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "policy 2",
        **dict(
            get_do_nothing_dict(),
            **{"4_RfR 0": 1, "4_RfR 1": 1, "4_RfR 2": 1, "A.5_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "policy 3",
        **dict(
            get_do_nothing_dict(),
            **{"1_RfR 0": 1, "2_RfR 1": 1, "3_RfR 2": 1, "A.3_DikeIncrease 0": 5}
        )
    ),
]

In [ ]:
# pass the policies list to EMA workbench experiment runs
n_scenarios = 100
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, policies)

In [ ]:
experiments, outcomes = results

In [ ]:
# only works because we have scalar outcomes
pd.DataFrame(outcomes)

GSA without Policies

In [ ]:
from ema_workbench import (
    perform_experiments, Samplers, Policy
)
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from SALib.analyze import sobol
from problem_formulation import get_model_for_problem_formulation
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from ema_workbench import MultiprocessingEvaluator

# Detect number of physical/logical cores
import multiprocessing

total_cores = multiprocessing.cpu_count()  # e.g., 16 logical cores for Ryzen 7700X
desired_cores = total_cores // 2  # use 50% of your CPU (adjust as needed)

N = 512  # Sobol sample base size
with MultiprocessingEvaluator(model, n_processes=desired_cores) as evaluator:
    experiments, outcomes = evaluator.perform_experiments(
        scenarios=N,
        policies=[zero_policy],
        uncertainty_sampling=Samplers.SOBOL
    )

In [ ]:
outcomes

In [ ]:
from SALib.analyze import sobol
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
import numpy as np

# Step 1: Get the SALib problem setup
problem = get_SALib_problem(model.uncertainties)
expected_len = N * (len(problem['names']) + 2)

# Step 2: Initialize result dictionaries
sobol_results_mean = {}
sobol_results_std = {}
sobol_results_final = {}

# Step 3: Loop through outcomes and apply reduction
for outcome_name, values in outcomes.items():
    arr = np.asarray(values)[:expected_len]

    if arr.ndim == 1:
        # Already scalar
        y_mean = arr
        y_std = np.zeros_like(arr)
        y_final = arr
    elif arr.ndim == 2:
        # 2D array like (20480, 3)
        y_mean = arr.mean(axis=1)
        y_std = arr.std(axis=1)
        y_final = arr[:, -1]
    else:
        print(f"⚠️ Skipping unsupported shape: {outcome_name}, shape={arr.shape}")
        continue

    # Run Sobol analysis for each reduction
    sobol_results_mean[outcome_name] = sobol.analyze(problem, y_mean, calc_second_order=False)
    sobol_results_std[outcome_name] = sobol.analyze(problem, y_std, calc_second_order=False)
    sobol_results_final[outcome_name] = sobol.analyze(problem, y_final, calc_second_order=False)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Extract parameter names from the problem
parameter_names = problem['names']

# Convert a Sobol result dictionary into a tidy DataFrame
def sobol_to_dataframe(sobol_dict, parameter_names, label):
    records = []
    for outcome_name, Si in sobol_dict.items():
        for i, param in enumerate(parameter_names):
            records.append({
                'Outcome': outcome_name,
                'Parameter': param,
                'S1': Si['S1'][i],
                'ST': Si['ST'][i],
                'S1_conf': Si['S1_conf'][i],
                'ST_conf': Si['ST_conf'][i],
                'Reduction': label
            })
    return pd.DataFrame(records)

# Convert all three dictionaries to DataFrames
df_mean = sobol_to_dataframe(sobol_results_mean, parameter_names, 'Mean')
df_std = sobol_to_dataframe(sobol_results_std, parameter_names, 'Std')
df_final = sobol_to_dataframe(sobol_results_final, parameter_names, 'Final')

# Combine them all into one long-form DataFrame
df_all = pd.concat([df_mean, df_std, df_final], ignore_index=True)

# Plotting function for heatmaps
def plot_sobol_heatmap(df, reduction_type):
    pivot = df[df['Reduction'] == reduction_type].dropna(subset=["ST"])\
        .pivot(index='Outcome', columns='Parameter', values='ST')

    if pivot.empty:
        print(f"⚠️ No valid data to plot for reduction type: {reduction_type}")
        return

    plt.figure(figsize=(14, max(6, len(pivot) * 0.4)))
    sns.heatmap(pivot, annot=True, fmt=".2f", cmap='YlGnBu', cbar_kws={'label': 'Total Sobol Index (ST)'})
    plt.title(f"Sobol ST Heatmap ({reduction_type})")
    plt.ylabel("Outcome")
    plt.xlabel("Input Parameter")
    plt.tight_layout()
    plt.show()

# Plot all three heatmaps
plot_sobol_heatmap(df_all, 'Mean')
plot_sobol_heatmap(df_all, 'Std')
plot_sobol_heatmap(df_all, 'Final')